In [1]:
import cv2
import glob
import numpy as np

In [2]:
?np.mgrid

In [3]:
column, row = 4, 5
objp = np.zeros(shape=(column*row, 3), dtype=np.float32)
objp[:, :2] = np.mgrid[0:row, 0:column].T.reshape(-1, 2)

In [4]:
objps = []
imgps = []

In [5]:
pic_dir = '/media/commaai-03/Data/tmp2/calibrationdata'

In [6]:
for img in glob.glob(pic_dir + '/*.png'):
    im = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    pattern_found, corners = cv2.findChessboardCorners(im, 
                                                       (row, column))
    if pattern_found:
        objps.append(objp)
        imgps.append(corners)


In [11]:
print(objps[0])
imgps[0]

[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [4. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [3. 1. 0.]
 [4. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [3. 2. 0.]
 [4. 2. 0.]
 [0. 3. 0.]
 [1. 3. 0.]
 [2. 3. 0.]
 [3. 3. 0.]
 [4. 3. 0.]]


array([[[143.      , 175.      ]],

       [[136.5     , 153.      ]],

       [[131.5388  , 129.20465 ]],

       [[120.97319 , 104.76247 ]],

       [[115.38538 ,  79.41136 ]],

       [[167.45128 , 173.69576 ]],

       [[160.64548 , 150.40016 ]],

       [[153.95816 , 128.44814 ]],

       [[147.23663 , 103.25036 ]],

       [[139.4187  ,  78.478134]],

       [[189.9117  , 170.63808 ]],

       [[183.09372 , 148.68768 ]],

       [[176.49828 , 126.655266]],

       [[171.55902 , 102.72516 ]],

       [[164.5076  ,  74.280815]],

       [[212.64047 , 169.42302 ]],

       [[206.76158 , 146.71643 ]],

       [[201.18282 , 123.40666 ]],

       [[195.4485  ,  97.27209 ]],

       [[189.      ,  73.5     ]]], dtype=float32)

In [12]:
matrix_init = np.zeros((3, 3), np.float32)
matrix_init[0][0] = 310.940583445
matrix_init[0][2] = 320.0
matrix_init[1][1] = 310.940583445
matrix_init[1][2] = 320.0
matrix_init[2][2] = 1.0

In [13]:
pattern_found, K, D, rvecs, tvecs = cv2.calibrateCamera(objps, 
                                                        imgps,
                                                        im.shape[::-1],
                                                        matrix_init,
                                                        None,
                                                        flags=1+8+16384)

In [ ]:
im.shape[::-1]

In [14]:
K

array([[508.25338381,   0.        , 342.75658344],
       [  0.        , 506.64650779, 237.4152401 ],
       [  0.        ,   0.        ,   1.        ]])

In [15]:
D

array([[  4.21419293, -39.42400852,   0.        ,   0.        ,
         61.8680341 ,   4.15997269, -39.17858003,  61.57940946,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ]])

### Extrinsic

In [16]:
import json

In [17]:
json_file = '/media/commaai-03/Data/tmp2/c1226/bg/c0.json'
bg = '/media/commaai-03/Data/tmp2/c1226/bg/c0.jpg'

with open(json_file, 'r') as f:
    c0_info = json.load(f, encoding='utf-8')

In [18]:
_objPoints = {}
for i in range(5):
    for j in range(5):
        #
        label1 = 'p%d-p%d'%(i,j)
        label2 = 'p%d-n%d'%(i,j)
        label3 = 'n%d-p%d'%(i,j)
        label4 = 'n%d-n%d'%(i,j)
        #
        _objPoints[label1] = [i*0.5, j*0.5, 0]
        _objPoints[label2] = [i*0.5, -j*0.5, 0]
        _objPoints[label3] = [-i*0.5, j*0.5, 0]
        _objPoints[label4] = [-i*0.5, -j*0.5, 0]

In [19]:
_objPoints.items()[:5]

[('n3-p3', [-1.5, 1.5, 0]),
 ('n3-p2', [-1.5, 1.0, 0]),
 ('n3-p1', [-1.5, 0.5, 0]),
 ('n3-p0', [-1.5, 0.0, 0]),
 ('n3-p4', [-1.5, 2.0, 0])]

In [20]:
c0_info.keys()

[u'imagePath',
 u'imageData',
 u'shapes',
 u'version',
 u'flags',
 u'fillColor',
 u'lineColor',
 u'imageWidth',
 u'imageHeight']

In [21]:
objps = []
imgps = []
for shape in c0_info['shapes']:
    point = shape['points'][0]
    objP = _objPoints[shape['label']]
    imgps.append(point)
    objps.append(objP)

In [23]:
imgps = np.array(imgps, dtype=np.float32)
objps = np.array(objps, dtype=np.float32)

In [45]:
ret, rvec, tvec, inliers \
    = cv2.solvePnPRansac(objps, 
                         imgps, 
                         K, 
                         D)

In [46]:
rvec

array([[ 2.19221672],
       [ 2.14301657],
       [-0.15368367]])

In [47]:
tvec

array([[-0.47887079],
       [-0.96232791],
       [ 3.07350631]])

In [124]:
dst, jac = cv2.Rodrigues(rvec)

array([[ 0.02140315,  0.99954723, -0.02114796],
       [ 0.99233646, -0.02381407, -0.12124869],
       [-0.12169741, -0.01839079, -0.99239686]])

In [116]:
o = np.float32([[1.0, 0.5, 0]]).reshape(-1,3)
imgpts, jac = cv2.projectPoints(o, rvec, tvec, K, D)
x, y = int(tuple(imgpts[0].ravel())[0]), int(tuple(imgpts[0].ravel())[1])
_img = cv2.imread(bg)
p_img = cv2.circle(_img, (x, y), 3, (0,0,255), -1)
while 1:
    cv2.imshow('P', p_img)
    k = cv2.waitKey(1)
    if k == 27:
        cv2.destroyAllWindows()
        break